## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<div style="border:solid green 2px; padding: 20px">

<h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Привет! Поздравляю тебя с первым твоим проектом и спасибо, что вовремя сдал задание:) Ты проделал большую работу. Далее в файле мои комментарии ты сможешь найти в ячейках, аналогичных данной ( если рамки комментария зелёные - всё сделано правильно; жёлтые - есть замечания, но не критично; красные - нужно переделать). Не удаляй эти комментарии и постарайся учесть их в ходе выполнения проекта. 

</div>

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
print(df.info())
print(df.head(10))
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None
   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1  

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Вывод

Изучив материал, видно что некорректные данные присутствуют в некоторых столбцах. Таких как стаж работы. Он не может быть отрицательным.

<div style="border:solid  green  2px; padding: 20px">

<h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>



Хорошо

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
#Есить пропуски в total_income и days_employed. Убираем по средством присваивания пропущенным значениям среднее.
df.info()

df['education'] = df['education'].str.lower()   #К нижнему регистру

#две медианы (высшее, среднее)
secondary_education_income_median = df.loc[df['education'] == 'среднее']['total_income'].median()
higher_education_income_median = df.loc[df['education'] == 'высшее']['total_income'].median()

#Функция
def total_income_fill(row):               
    education = row['education']                    #Функция которая обрабатывает логику заполнения пропусков
    if education == 'высшее':                       #если ученая степень, то возвращает тоже высшее 
        return higher_education_income_median       #(т.к. ученых там 6 человек и под них медиану смысла нет выводить)
    elif education == 'ученая степень':             #во всех остальных случаях возвращает медиану среднего образования
        return higher_education_income_median
    return secondary_education_income_median

#а потом эту функцию через метод apply засунуть в fillna
df['total_income'] = df['total_income'].fillna(df.apply(total_income_fill, axis=1))

df['days_employed'] = df['days_employed'].fillna(0)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_stat

### Вывод

Заменяем пропуски в зависимости от образования. Предварительно вычислив среднюю зп для каждого образования. После заменяем для каждлго своим средним.Есть и люди с ученой степенью.Если ученая степень, то возвращает тоже высшее (т.к. ученых там 6 человек и под них медиану смысла нет выводить).


<div style="border:solid green 2px; padding: 20px">

<h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Отлично

### Замена типа данных

In [4]:
#ВЕРНУТЬСЯ


def change(row):
    if row < 0:
        return int(-row)
    else:
        return int(row)
df['days_employed'] = df['days_employed'].apply(change)
df['total_income'] = df['total_income'].apply(change)



### Вывод

Заменяем значения на положительные в столбце "days_employed"
Делается через функцию. Что бы преобразовать days_employed в целые и положительные,
будем прогонять кажный через функцию, которыя будт заменять значение на правильное в зависимости от него.
(например если оно отрицательное и не целое, его преобразовывают в положительное по модулю а потом сокращают до целого)
Данный способы был выбан, так как просто и понятно меняет любое отрицательное числа на положительное и любое дробное на целое.

<div style="border:solid  orange  2px; padding: 20px">

<h1 style="color: orange ; margin-bottom:20px">Комментарий наставника</h1>
Здесь лучше бы подошёл метод astype, можешь почитать о нём в документации к библиотеке
</div>

### Лемматизация

In [5]:
# заменить значения в purpose предварительно изменив весь регистр на нижний
df['purpose'] = df['purpose'].str.lower()
df['family_status'] = df['family_status'].str.lower()

#lemmas = df['purpose'].apply(m.lemmatize)
#lemmas.value_counts()

from pymystem3 import Mystem
m = Mystem()
def lemm(row):
    lemmas = m.lemmatize(row)
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    elif ('жилье' in lemmas) or ('недвижимость' in lemmas):
        return 'жильё'
    elif 'образование' in lemmas:
        return 'образование'
    else:
        return row
df['word_lemmas'] = df['purpose'].apply(lemm)
#print(df['purpose'].value_counts())
print(df['word_lemmas'].value_counts())

жильё          10840
автомобиль      4315
образование     4022
свадьба         2348
Name: word_lemmas, dtype: int64


### Вывод

В столбце 'purpose' есть одинаковые по значимости строчки. Такие как покупка машины и автомобиль.
Их приводм к одному значению и наименованию. Получается 4 категории с которыми мы будем работать.


<div style="border:solid green 2px; padding: 20px">

<h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Всё верно

### Обработка дубликатов

In [6]:
#Метод drop_duplicates() для удаления дубликатов df.drop_duplicates().reset_index(drop=True)
# аргумент drop со значением True,
# чтобы не создавать столбец со
# старыми значениями индексов'''При вызове метода drop_duplicates()
#вместе с повторяющимися строкамиудаляются их индексы, поэтомуиспользуется с методом reset_index()

print('\nКол-во дубликатов до - ', df.duplicated().sum(), sep='')
df = df.drop_duplicates().reset_index(drop=True)
print('\nКол-во дубликатов после - ', df.duplicated().sum(), sep='')


Кол-во дубликатов до - 71

Кол-во дубликатов после - 0


### Вывод

Удалили дубликаты и вернули нормальные индексы

<div style="border:solid  green  2px; padding: 20px">

<h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>


Дубликаты обработаны и найдены верно, отлично сделано


### Категоризация данных

In [7]:
#ввожу условную классификацию 0<50k<100k<200k<300k<300k+ - низкий, предсредний, средний, высокий, сверхвысокий.
#Функция для ответа на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок"

def income(level):
    if level < 50000:  
        return 'низкий'
    if level >= 50000 and level <= 100000:  
        return 'предсредний' 
    if level >= 200000 and level <= 200000:  
        return 'средний'
    if level >= 200000 and level <= 300000:
        return 'высокий'
    if level > 300000:
        return 'сверхвысокий'
df['income_level'] = df['total_income'].apply(income)




#Функция для категоризации детей, что в последствии поможет ответить на вопрос:
#"Есть ли зависимость между наличием детей и возвратом кредита в срок?"

def child(row):
     if row == 0:
        return 'нет детей'
     return 'есть дети'

df['children_new'] = df['children'].apply(child)
#df['children_new'].value_counts()
#Потом сводная таблица будет иметь вид:
#df_piv_child = df.pivot_table(index=['children_new'], columns='debt', values='children', aggfunc=')#


#Разделяем семейное положение на 2 части: Есть семья или нет
def family(stat):
    if stat == 0:
        return 'нет семьи'
    return 'есть семья'

df['fsd_new'] = df['family_status_id'].apply(family)
#df['children_new'].value_counts()

### Вывод

Ввели условную классификацию 0<50k<100k<200k<300k<300k+ - низкий, предсредний, средний, высокий, сверхвысокий. Создали функцию, которая обрабатывает total_income и сформируем столбец 'income_level'.

<div style="border:solid green 2px; padding: 20px">

<h1 style="color:green; margin-bottom:20px">Комментарий наставника</h1>

Хорошо, так держать)

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [8]:
df_piv_child = df.pivot_table(index=['children_new'], columns='debt', values='children', aggfunc='count') #создаем сводную таблицу
df_piv_child['ratio'] = df_piv_child[1]/(df_piv_child[0]+df_piv_child[1])*100 #создаем в таблице новый столбец считающий соотношение 1 и 2 столбца для ответа на поставненный вопрос
df_piv_child

debt,0,1,ratio
children_new,,,
есть дети,6685,678,9.208203
нет детей,13028,1063,7.543822


### Вывод

Делаем вывод, что менее рискованные кредиты берут семьи без детей - не надо их содержать.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [9]:
df_piv_family = df.pivot_table(index=['fsd_new'], columns='debt', values='family_status', aggfunc='count') #создаем сводную таблицу
df_piv_family['ratio'] = df_piv_family[1]/(df_piv_family[0]+df_piv_family[1])*100 #создаем в таблице новый столбец считающий соотношение 1 и 2 столбца для ответа на поставненный вопрос
df_piv_family

debt,0,1,ratio
fsd_new,,,
есть семья,8305,810,8.886451
нет семьи,11408,931,7.545182


### Вывод

Делаем вывод, что менее рискованные кредиты берут люди без семьи.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
df_piv_income_level = df.pivot_table(index=['income_level'], columns='debt', values='total_income', aggfunc='count')
df_piv_income_level['ratio'] = df_piv_income_level[1]/(df_piv_income_level[0]+df_piv_income_level[1])*100
df_piv_income_level

debt,0,1,ratio
income_level,,,
высокий,3332,252,7.031250
низкий,349,23,6.182796
предсредний,3760,331,8.090931
сверхвысокий,1376,106,7.152497


### Вывод

Делаем вывод, что менее рискованные кредиты берут люди с низкой и средней заработной платой.

- Как разные цели кредита влияют на его возврат в срок?

In [11]:
df_piv_purpose = df.pivot_table(index=['word_lemmas'], columns='debt', values='purpose', aggfunc='count') #создаем сводную таблицу
df_piv_purpose['-ratio'] = df_piv_purpose[1]/(df_piv_purpose[0]+df_piv_purpose[1])*100 #создаем в таблице новый столбец считающий соотношение 1 и 2 столбца для ответа на поставненный вопрос
df_piv_purpose

debt,0,1,-ratio
word_lemmas,,,
автомобиль,3903,403,9.359034
жильё,10029,782,7.233373
образование,3643,370,9.220035
свадьба,2138,186,8.003442


### Вывод

Делаем вывод, что менее рискованные кредиты - это недвижимость и жильё (это одно и то же).
Риск повышается при кредите на свадьбу - появляется вероятность неудачных браков.
И наиболее рискованные - на автомобиль - вероятность дальнейших поломок, ДТП, краж
И образование - проблемы с поиском работы

<div style="border:solid  green  2px; padding: 20px">

<h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>



Все выводы верны, код написан правильно, радует, что используешь сводные таблицы)


### Шаг 4. Общий вывод

После обработки пропусков в столбцах "total_income" и "days_employed" приступаем к замене типов данных в столбце "days_employed", а именно превращаем все числа в целые и положительные, т.к стаж не может быть отрицательным.
Затем делаем лемматизацию столбца "purpose" для того что бы вывести стова которые помогут нам в дальнейшем сделать выводы.
После чего удаляем все дубликаты в таблице, что бы не было не точных значений.
Затем приступаем к категоризации, где котегоризируем те слова, которые нам пригодятся для решения поставленных задач. 
Далее делаем сводные таблици для каждой задачи, что бы ответить на нее.
Из сводных таблиц делаем выводы, что:
Менее рискованные кредиты берут семьи без детей - не надо их содержать.
Менее рискованные кредиты берут люди без семьи.
Менее рискованные кредиты берут люди с низкой и средней заработной платой.
Менее рискованные кредиты - это недвижимость и жильё.
Риск повышается при кредите на свадьбу - появляется вероятность неудачных браков.
И наиболее рискованные - на автомобиль - вероятность дальнейших поломок, ДТП, краж
И образование - проблемы с поиском работы

<div style="border:solid  green  2px; padding: 20px">

<h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>

Конкретный и достаточно подробный общий вывод, супер. Как совет на будущее - можешь в вывод так же добавлять и цифры для большей наглядности результатов работы




### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.

<div style="border:solid  orange  2px; padding: 20px">

<h1 style="color: orange ; margin-bottom:20px">Комментарий наставника</h1>

Работа хорошая, но стоит исправить обозначенные неточности, буду ждать исправленный вариант


<div style="border:solid  green  2px; padding: 20px">

<h1 style="color: green ; margin-bottom:20px">Комментарий наставника</h1>

#### Код

Всё отлично. Из того, что очень порадовало - соблюдена структура проекта, шаги из задания обозначены и выполнены последовательно, код написан аккуратно, используются комментарии к коду, быстро можно понять, какие операции выполняют сложные конструкции, названия переменных передают смысл операций. В качестве совета предлагаю глубже изучить и начать применять конструкцию try-except в решении задачи — это улучшит отказоустойчивость кода и обезопасит код от поломок в будущем.

#### Выводы

У тебя отлично получается анализировать сложные данные, выдвигать корректные гипотезы и проверять свои выводы на возможность соответствия реальности. Видно глубокое понимание сути проведённого анализа. Было очень интересно проверять твой проект и следить за твоей мыслью, так держать!)
